# First Spark Streaming Example
_____

In [1]:
import findspark

findspark.init('/Users/neerajpadarthi/Neeraj/spark-2.3.0-bin-hadoop2.7')

# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from pyspark.sql import Row
import re

# Can only run this once. restart your kernel for any errors.
sc = SparkContext.getOrCreate()
# sc1 = SparkSession.builder.appName("Hive TST").enableHiveSupport().getOrCreate()



ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

socket_stream = ssc.socketTextStream("127.0.0.1", 5551)

lines = socket_stream.window( 20,20 )

from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

fields = ("tweet" )
Tweet1 = namedtuple( 'Tweet', fields )

# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) 
  .filter( lambda word: word.lower().startswith("#") ) 
  .map( lambda word: ( word.lower(), 1 ) ) 
  .reduceByKey( lambda a, b: a + b ) 
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) 
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) 
  .limit(10).registerTempTable("tweets") ) ) 

# lines.pprint()

__________
### Run the TweetRead.py file at this point
__________

In [2]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
# Only works for Jupyter Notebooks!
%matplotlib inline 

ssc.start()

In [3]:
count = 0
while count < 10:
    time.sleep(5)
    sqlContext.sql( 'Select tag, count from tweets').show()
    count=count+1

+------------------+-----+
|               tag|count|
+------------------+-----+
|     #seagames2019|    1|
|     #wearebadgers|    1|
|     #gobblegobble|    1|
|             #coys|    1|
|            #ihsaa|    1|
|             #thfc|    1|
|      #bluedotsale|    1|
|#happythanksgiving|    1|
|#dumptrump2020does|    1|
|              #nfl|    1|
+------------------+-----+

+------------------+-----+
|               tag|count|
+------------------+-----+
|     #seagames2019|    1|
|     #wearebadgers|    1|
|     #gobblegobble|    1|
|             #coys|    1|
|            #ihsaa|    1|
|             #thfc|    1|
|      #bluedotsale|    1|
|#happythanksgiving|    1|
|#dumptrump2020does|    1|
|              #nfl|    1|
+------------------+-----+

+------------------+-----+
|               tag|count|
+------------------+-----+
|     #seagames2019|    1|
|     #wearebadgers|    1|
|     #gobblegobble|    1|
|             #coys|    1|
|            #ihsaa|    1|
|             #thfc|    1|

In [4]:
ssc.stop()